In [22]:
import pandas as pd
import glob as glob
import csv
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import isodate
import re

In [ ]:
## Unique columns in each CSV file ## 
def read_csv_headers(file_path):
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Read the first line
    return set(headers)

# Directory containing your CSV files
directory = '../data/Eye-tracking Output/raw_data'

# Read headers from the first file (1.csv)
base_headers = read_csv_headers(os.path.join(directory, '1.csv'))

# Compare headers from all other files to the first file
for i in range(2, 26):
    file_path = os.path.join(directory, f'{i}.csv')
    if os.path.exists(file_path):
        current_headers = read_csv_headers(file_path)
        if current_headers != base_headers:
            print(f"\n{i}.csv differs from 1.csv:")
            print("Missing columns in 1.csv:", current_headers - base_headers)
            print("Additional columns in 1.csv:", base_headers - current_headers)
    else:
        print(f"{i}.csv does not exist in the directory.")


In [ ]:
## Checking all files which contain the same column names ##

# Define the directory containing the CSV files (using the provided path by the user)
directory = '../data/Eye-tracking Output/raw_data'

# Initialize an empty list to store the column names from each file
all_columns = []

# Iterate over the file numbers and collect their column names
for i in range(1, 26):
    file_path = os.path.join(directory, f'{i}.csv')
    try:
        # Read just the first row to get the column names
        df = pd.read_csv(file_path, nrows=0)
        # Append the column names to the list
        all_columns.append(set(df.columns))
    except FileNotFoundError:
        print(f"File {i}.csv not found.")
    except Exception as e:
        print(f"An error occurred while processing {i}.csv: {e}")

# Use set intersection to find the common columns across all files
common_columns = set.intersection(*all_columns) if all_columns else set()

# Convert the common columns set to a sorted list
common_columns_list = sorted(list(common_columns))
common_columns_list

## saving the column names into a file ## 
def read_csv_headers(file_path):
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Read the first line
    return headers

# Directory containing your CSV files
directory = '../data/Eye-tracking Output/raw_data'

# Set to store unique column names
unique_columns = set()

# Read headers from all files and update the set of unique columns
for i in range(1, 26):
    file_path = os.path.join(directory, f'{i}.csv')
    if os.path.exists(file_path):
        headers = read_csv_headers(file_path)
        unique_columns.update(headers)
    else:
        print(f"{i}.csv does not exist in the directory.")

# Path for the new CSV file to save unique columns
output_file = os.path.join(directory, 'unique_columns.csv')

# Write unique column names to the new CSV file
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for column in sorted(unique_columns):
        writer.writerow([column])

print(f"Unique columns have been saved to {output_file}")

In [ ]:
## Checking if all ET expirements had different videos ## 
def read_stimulus_values(file_path):
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        return set(row['Stimulus'] for row in reader)

# Directory containing your CSV files
directory = '../data/Eye-tracking Output/raw_data'

# Store the stimulus values for each file
stimulus_dict = {}
for i in range(1, 26):
    file_path = os.path.join(directory, f'{i}.csv')
    if os.path.exists(file_path):
        stimulus_dict[f'{i}.csv'] = read_stimulus_values(file_path)
    else:
        print(f"{i}.csv does not exist in the directory.")

# Compare the stimulus sets
unique_sets = {}
for filename, stimuli in stimulus_dict.items():
    if stimuli not in unique_sets.values():
        unique_sets[filename] = stimuli

# Print results
print("Unique Stimulus Sets:")
for filename in unique_sets:
    print(f"- {filename}")

print("\nFiles with Same Stimulus Sets:")
for file1, stimuli1 in stimulus_dict.items():
    same_sets = [file2 for file2, stimuli2 in stimulus_dict.items() if stimuli1 == stimuli2 and file1 != file2]
    if same_sets:
        print(f"{file1} has the same stimulus set as: {', '.join(same_sets)}")

In [53]:
## Concatting data into single CSV file ##
# Path to the directory containing CSV files
path = "../data/Eye-tracking Output/raw_data/*.csv"

# Using glob to find all csv files in the directory
file_list = glob.glob(path)

# Define data types for columns with mixed types
dtype_dict = {str(i): str for i in range(15, 43)}
dtype_dict['Participant'] = str  # Ensure Participant is also read as string

# Reading all csv files and appending them to a list
list_ = []
for file_ in file_list:
    df = pd.read_csv(file_, dtype=dtype_dict, index_col=None, header=0)
    list_.append(df)

# Concatenating all csv files into one dataframe
df = pd.concat(list_, axis=0, ignore_index=True)

# Saving the dataframe as a csv file
df.to_csv("../data/Eye-tracking Output/combined.csv", index=False)

/var/folders/94/903y39c95x3g49g0_7l22jn00000gn/T/ipykernel_86845/4294307232.py:15: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_, dtype=dtype_dict, index_col=None, header=0)


In [55]:
## Checking how many participants in raw csv files compared to combined ##
# Directory containing your CSV files
directory = '../data/Eye-tracking Output/raw_data'

def count_unique_participants(directory, participant_id_column):
    participant_ids = set()
    for i in range(1, 26):
        file_path = os.path.join(directory, f'{i}.csv')
        try:
            with open(file_path, 'r') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    participant_ids.add(row[participant_id_column])
        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}")

    return len(participant_ids)

participant_id_column = 'Participant' 
num_participants = count_unique_participants(directory, participant_id_column)
print(f"Total number of different participants across all CSV files: {num_participants}")

# checking number of participants in the combined csv
df = pd.read_csv('../data/Eye-tracking Output/combined.csv')
n_participants = len(pd.unique(df['Participant']))
print("dataset has", n_participants, "unique participants")

# NOTE ved fandeme ikke lige hvorfor den siger, der er 91 unique participants

Total number of different participants across all CSV files: 59


/var/folders/94/903y39c95x3g49g0_7l22jn00000gn/T/ipykernel_86845/482206397.py:26: DtypeWarning: Columns (7,12,14,15,16,19,20,21,22,26,27,28,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/Eye-tracking Output/combined.csv')


dataset has 91 unique participants


In [24]:
## Cleaning the data ## 
# Load the main dataset
file_path = '../data/Eye-tracking Output/combined.csv'
df = pd.read_csv(file_path)

# Replace the last colon with a period to properly format the milliseconds
df['Time of Day [h:m:s:ms]'] = df['Time of Day [h:m:s:ms]'].str.rsplit(':', 1).str.join('.')

# Convert "Time of Day [h:m:s:ms]" to timedelta
df['Time of Day [h:m:s:ms]'] = pd.to_timedelta(df['Time of Day [h:m:s:ms]'])

# Sort by "Participant", "Trial", and "Stimulus"
df.sort_values(by=['Participant', 'Trial', 'Stimulus'], inplace=True)

# Subtract the first time for each group from all the times in that group
df['Time.s'] = df.groupby(['Participant', 'Trial', 'Stimulus'])['Time of Day [h:m:s:ms]'].transform(lambda x: x - x.iloc[0])

df['Time.s'] = df['Time.s'].apply(lambda x: x.total_seconds())


# Keep only relevant columns
relevant_columns = [
    'Participant', 'Trial', 'Stimulus', 'Time.s', 'RecordingTime [ms]',
    'Export Start Trial Time [ms]', 'Export End Trial Time [ms]',
    'Point of Regard Left X [px]', 'Point of Regard Left Y [px]',
    'Point of Regard Right X [px]', 'Point of Regard Right Y [px]',
    'Pupil Diameter Left [mm]', 'Pupil Diameter Right [mm]',
    'Tracking Ratio [%]', 'Category Left', 'Category Right'
]
df = df[relevant_columns]

# Drop rows with missing values
df.dropna(inplace=True)

# Remove unidentified participants
df = df[~df['Participant'].isin(["Unidentified(Neg)", "Unidentified(Pos)"])]

# Convert 'Participant' to numeric if necessary
df['Participant'] = pd.to_numeric(df['Participant'], errors='coerce')

# Load the metadata
df_metadata = pd.read_csv('../data/Metadata_Participants.csv')

# Merge the metadata with the main dataframe
df = pd.merge(df, df_metadata, left_on='Participant', right_on='ParticipantID', how='left')

# Drop the 'ParticipantID' column
df.drop('ParticipantID', axis=1, inplace=True)


# Save the cleaned data
output_file = '../data/Eye-tracking Output/cleaned_data.csv'
df.to_csv(output_file, index=False)

print("Data cleaning completed and saved to", output_file)


/var/folders/94/903y39c95x3g49g0_7l22jn00000gn/T/ipykernel_97442/2136264761.py:4: DtypeWarning: Columns (7,12,14,15,16,19,20,21,22,26,27,28,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/94/903y39c95x3g49g0_7l22jn00000gn/T/ipykernel_97442/2136264761.py:7: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['Time of Day [h:m:s:ms]'] = df['Time of Day [h:m:s:ms]'].str.rsplit(':', 1).str.join('.')


Data cleaning completed and saved to ../data/Eye-tracking Output/cleaned_data.csv
